# Устанавливаем пайтон клиент для инфера тритон-сервера

In [2]:
!pip install tritonclient[all]

Looking in indexes: https://artifactory.s.o3.ru/artifactory/api/pypi/pypi-virtual/simple, https://pypi.org/simple


# Инитим тритон-клиент

In [3]:
import tritonclient.grpc as grpcclient

triton_client = grpcclient.InferenceServerClient(url='localhost:14281')


In [4]:
triton_client.is_server_live()

True

In [7]:
triton_client.get_server_metadata()

name: "mcap_izhdanov_triton"
version: "2.46.0"
extensions: "classification"
extensions: "sequence"
extensions: "model_repository"
extensions: "model_repository(unload_dependents)"
extensions: "schedule_policy"
extensions: "model_configuration"
extensions: "system_shared_memory"
extensions: "cuda_shared_memory"
extensions: "binary_tensor_data"
extensions: "parameters"
extensions: "statistics"
extensions: "trace"
extensions: "logging"

## Загружаем модель

In [6]:
triton_client.get_model_repository_index()

models {
  name: "add_sub"
}

In [10]:
triton_client.load_model('add_sub')

# Создаём запрос для инфера модели через сервер

In [12]:
import numpy as np

data = np.random.random(4)
data

array([0.37641487, 0.65700403, 0.73188738, 0.09409728])

## Формируем входные и выходные тензоры

In [ ]:
inputs = []
inputs.append(grpcclient.InferInput('input0', [4], "FP32"))
inputs[0].set_data_from_numpy(np.array(data, dtype=np.float32))   

# always check your input data's shape before infer!
inputs[0].shape()

[4]

In [18]:
outputs = []
outputs.append(grpcclient.InferRequestedOutput('output0'))

## Производим инфер нашей модели

In [ ]:
# get response from model
results = triton_client.infer('add_sub',
                                inputs,
                                '1',
                                outputs)
# parse data from response
data2 = results.as_numpy('output0')
data2

array([0.14168815, 0.43165433, 0.5356592 , 0.0088543 ], dtype=float32)

## Смотрим на степень сжатия данных внутри ответа сервера

In [26]:
data**2==data2

array([False, False, False, False])

In [25]:
data**2, data2

(array([0.14168815, 0.4316543 , 0.53565913, 0.0088543 ]),
 array([0.14168815, 0.43165433, 0.5356592 , 0.0088543 ], dtype=float32))